In [2]:
from gurobipy import *

In [3]:
import json
with open('toy_instance.json') as f:
    json_instance = json.load(f)
    f.close()

In [4]:
json_instance

{'horizon': 5,
 'qualifications': ['A', 'B', 'C'],
 'staff': [{'name': 'Olivia',
   'qualifications': ['A', 'B', 'C'],
   'vacations': []},
  {'name': 'Liam', 'qualifications': ['A', 'B'], 'vacations': [1]},
  {'name': 'Emma', 'qualifications': ['C'], 'vacations': [2]}],
 'jobs': [{'name': 'Job1',
   'gain': 20,
   'due_date': 3,
   'daily_penalty': 3,
   'working_days_per_qualification': {'A': 1, 'B': 1, 'C': 1}},
  {'name': 'Job2',
   'gain': 15,
   'due_date': 3,
   'daily_penalty': 3,
   'working_days_per_qualification': {'A': 1, 'B': 2}},
  {'name': 'Job3',
   'gain': 15,
   'due_date': 4,
   'daily_penalty': 3,
   'working_days_per_qualification': {'A': 1, 'C': 2}},
  {'name': 'Job4',
   'gain': 20,
   'due_date': 3,
   'daily_penalty': 3,
   'working_days_per_qualification': {'B': 2, 'C': 1}},
  {'name': 'Job5',
   'gain': 10,
   'due_date': 5,
   'daily_penalty': 3,
   'working_days_per_qualification': {'C': 2}}]}

In [18]:
N = len(json_instance['staff'])
M = len(json_instance['jobs'])
K = len(json_instance['qualifications'])
T = int(json_instance['horizon'])

In [75]:
mapping_names = {}
for i in range(len(json_instance['staff'])):
    mapping_names[i] = json_instance['staff'][i]['name']
mapping_names

{0: 'Olivia', 1: 'Liam', 2: 'Emma'}

In [81]:
mapping_qualif = {v:k for (k,v) in enumerate(json_instance['qualifications'])}
mapping_qualif

{'A': 0, 'B': 1, 'C': 2}

In [29]:
import numpy as np
qualifications_mat = np.zeros((N,K))
for i in range(N):
    for qual_k in json_instance['staff'][i]['qualifications']:
        qualifications_mat[i,mapping_qualif[qual_k]]=1
qualifications_mat


array([[1., 1., 1.],
       [1., 1., 0.],
       [0., 0., 1.]])

In [49]:
vacation_mat = np.ones((N,T))
for i in range(N):
    for t in json_instance['staff'][i]['vacations']:
        vacation_mat[i,t-1]=0
vacation_mat

array([[1., 1., 1., 1., 1.],
       [0., 1., 1., 1., 1.],
       [1., 0., 1., 1., 1.]])

In [31]:
wd_per_qual_mat = np.zeros((M,K))
for j in range(M):
    for qual,wd in json_instance['jobs'][j]['working_days_per_qualification'].items():
        wd_per_qual_mat[j,mapping_qualif[qual]] = wd
wd_per_qual_mat


array([[1., 1., 1.],
       [1., 2., 0.],
       [1., 0., 2.],
       [0., 2., 1.],
       [0., 0., 2.]])

In [46]:
gains_vect = np.zeros((M,))
for j in range(M):
    gains_vect[j] = json_instance['jobs'][j]['gain']
gains_vect

array([20., 15., 15., 20., 10.])

In [47]:
penalties_vect = np.zeros((M,))
for j in range(M):
    penalties_vect[j] = json_instance['jobs'][j]['daily_penalty']
penalties_vect

array([3., 3., 3., 3., 3.])

In [50]:
due_dates_mat = np.zeros((M,T))
for j in range(M):
    for t in range(json_instance['jobs'][j]['due_date']-1,T):
        due_dates_mat[j,t] = 1
due_dates_mat

array([[0., 0., 1., 1., 1.],
       [0., 0., 1., 1., 1.],
       [0., 0., 0., 1., 1.],
       [0., 0., 1., 1., 1.],
       [0., 0., 0., 0., 1.]])

In [87]:
# Instanciation du modèle
m = Model("PL modelling CompuOpti")

# Vecteur de variables
x = m.addVars(N,M,K,T,name="x", vtype=GRB.BINARY)
r = m.addVars(M,T, name="r", vtype=GRB.BINARY)
s = m.addVars(N,M, name="s", vtype=GRB.BINARY)
b = m.addVars(M,T, name="b", vtype=GRB.BINARY)
y = m.addVar(name="y",vtype=GRB.INTEGER)
z = m.addVar(name="z",vtype=GRB.INTEGER)

m.update()

m.addConstrs((x[i,j,k,t] <= qualifications_mat[i,k] for i in range(N) for j in range(M) for k in range(K) for t in range(T)), name = "qualification du personnel")
m.addConstrs((quicksum([x[i,j,k,t] for j in range(M) for k in range(K)]) <= 1 for i in range(N) for t in range(T)),name="unicité de l'affectation quotidienne du personnel")
m.addConstrs((x[i,j,k,t] <= vacation_mat[i,t] for i in range(N) for j in range(M) for k in range(K) for t in range(T)), name = "congés")
m.addConstrs(( quicksum([x[i,j,k,tau] for i in range(N) for tau in range(t+1)]) >= r[j,t] * wd_per_qual_mat[j,k] for j in range(M) for k in range(K) for t in range(T)),name="couverture des qualifications")
m.addConstrs(( quicksum([x[i,j,k,tau] for i in range(N) for tau in range(t+1)]) <= wd_per_qual_mat[j,k] for j in range(M) for k in range(K) for t in range(T)),name="unicité de la réalisation d'un projet")
m.addConstrs((b[j,t] >= r[j,t] for j in range(M) for t in range(T)),name="début d'un projet avant sa fin")
m.addConstrs((y >= quicksum([s[i,j] for j in range(M)]) for i in range(N)),name="linéarisation de l'objectif 2")
m.addConstrs((z >= quicksum([b[j,t] - r[j,t] for t in range(T)]) for j in range(M)), name = "linéarisation de l'objectif 3")
m.addConstrs((r[j,t+1] >= r[j,t] for j in range(M) for t in range(T-1)),name="contrainte interne à la variable r")
m.addConstrs((b[j,t+1] >= b[j,t] for j in range(M) for t in range(T-1)),name="contrainte interne à la variable b")

m.setObjective(quicksum([r[j,T-1]*(gains_vect[j]-quicksum([penalties_vect[j]*(1-r[j,t])*due_dates_mat[j,t] for t in range(T)])) for j in range(M)]), GRB.MAXIMIZE)

# Paramétrage (mode mute)
m.params.outputflag = 0

# Résolution du PL
m.optimize()

# Affichage en mode texte du PL
print(m.display())

print("La valeur optimale de l'objectif est f1 = {}".format(m.objVal))

for i in range(N):
    print('planning de {} : '.format(mapping_names[i]))
    for t in range(T):
        print('jour {} : '.format(t+1),end=' ')
        if vacation_mat[i,t]==0:
                    print('jour de congé')
        for j in range(M):
            for k in range(K):
                if x[i,j,k,t].X == 1 :
                    print('projet {}, compétence {}'.format(j+1,json_instance['qualifications'][k]))
                

None
La valeur optimale de l'objectif est f1 = 65.0
planning de Olivia : 
jour 1 :  projet 1, compétence A
jour 2 :  projet 2, compétence B
jour 3 :  projet 4, compétence B
jour 4 :  projet 3, compétence C
jour 5 :  projet 5, compétence C
planning de Liam : 
jour 1 :  jour de congé
jour 2 :  projet 1, compétence B
jour 3 :  projet 4, compétence B
jour 4 :  projet 3, compétence A
jour 5 :  projet 2, compétence B
planning de Emma : 
jour 1 :  projet 4, compétence C
jour 2 :  jour de congé
jour 3 :  projet 1, compétence C
jour 4 :  projet 3, compétence C
jour 5 :  projet 5, compétence C


In [89]:
# Instanciation du modèle
m2 = Model("PL modelling CompuOpti")

# Vecteur de variables
x = m2.addVars(N,M,K,T,name="x", vtype=GRB.BINARY)
r = m2.addVars(M,T, name="r", vtype=GRB.BINARY)
s = m2.addVars(N,M, name="s", vtype=GRB.BINARY)
b = m2.addVars(M,T, name="b", vtype=GRB.BINARY)
y = m2.addVar(name="y",vtype=GRB.INTEGER)
z = m2.addVar(name="z",vtype=GRB.INTEGER)

m2.update()

m2.addConstrs((x[i,j,k,t] <= qualifications_mat[i,k] for i in range(N) for j in range(M) for k in range(K) for t in range(T)), name = "qualification du personnel")
m2.addConstrs((quicksum([x[i,j,k,t] for j in range(M) for k in range(K)]) <= 1 for i in range(N) for t in range(T)),name="unicité de l'affectation quotidienne du personnel")
m2.addConstrs((x[i,j,k,t] <= vacation_mat[i,t] for i in range(N) for j in range(M) for k in range(K) for t in range(T)), name = "congés")
m2.addConstrs(( quicksum([x[i,j,k,tau] for i in range(N) for tau in range(t+1)]) >= r[j,t] * wd_per_qual_mat[j,k] for j in range(M) for k in range(K) for t in range(T)),name="couverture des qualifications")
m2.addConstrs(( quicksum([x[i,j,k,tau] for i in range(N) for tau in range(t+1)]) <= wd_per_qual_mat[j,k] for j in range(M) for k in range(K) for t in range(T)),name="unicité de la réalisation d'un projet")
m2.addConstrs((b[j,t] >= r[j,t] for j in range(M) for t in range(T)),name="début d'un projet avant sa fin")
m2.addConstrs((y >= quicksum([s[i,j] for j in range(M)]) for i in range(N)),name="linéarisation de l'objectif 2")
m2.addConstrs((z >= quicksum([b[j,t] - r[j,t] for t in range(T)]) for j in range(M)), name = "linéarisation de l'objectif 3")
m2.addConstrs((r[j,t+1] >= r[j,t] for j in range(M) for t in range(T-1)),name="contrainte interne à la variable r")
m2.addConstrs((b[j,t+1] >= b[j,t] for j in range(M) for t in range(T-1)),name="contrainte interne à la variable b")

m2.setObjective(y, GRB.MINIMIZE)

# Paramétrage (mode mute)
m2.params.outputflag = 0

# Résolution du PL
m2.optimize()

# Affichage en mode texte du PL
print(m2.display())

print("La valeur optimale de l'objectif est f2 = {}".format(m2.objVal))

for i in range(N):
    print('planning de {} : '.format(mapping_names[i]))
    for t in range(T):
        print('jour {} : '.format(t+1),end=' ')
        if vacation_mat[i,t]==0:
                    print('jour de congé')
        for j in range(M):
            for k in range(K):
                if x[i,j,k,t].X == 1 :
                    print('projet {}, compétence {}'.format(j+1,json_instance['qualifications'][k]))


None
La valeur optimale de l'objectif est f2 = 0.0
planning de Olivia : 
jour 1 :  jour 2 :  jour 3 :  jour 4 :  jour 5 :  planning de Liam : 
jour 1 :  jour de congé
jour 2 :  jour 3 :  jour 4 :  jour 5 :  planning de Emma : 
jour 1 :  jour 2 :  jour de congé
jour 3 :  jour 4 :  jour 5 :  

In [90]:
# Instanciation du modèle
m3 = Model("PL modelling CompuOpti")

# Vecteur de variables
x = m3.addVars(N,M,K,T,name="x", vtype=GRB.BINARY)
r = m3.addVars(M,T, name="r", vtype=GRB.BINARY)
s = m3.addVars(N,M, name="s", vtype=GRB.BINARY)
b = m3.addVars(M,T, name="b", vtype=GRB.BINARY)
y = m3.addVar(name="y",vtype=GRB.INTEGER)
z = m3.addVar(name="z",vtype=GRB.INTEGER)

m3.update()

m3.addConstrs((x[i,j,k,t] <= qualifications_mat[i,k] for i in range(N) for j in range(M) for k in range(K) for t in range(T)), name = "qualification du personnel")
m3.addConstrs((quicksum([x[i,j,k,t] for j in range(M) for k in range(K)]) <= 1 for i in range(N) for t in range(T)),name="unicité de l'affectation quotidienne du personnel")
m3.addConstrs((x[i,j,k,t] <= vacation_mat[i,t] for i in range(N) for j in range(M) for k in range(K) for t in range(T)), name = "congés")
m3.addConstrs(( quicksum([x[i,j,k,tau] for i in range(N) for tau in range(t+1)]) >= r[j,t] * wd_per_qual_mat[j,k] for j in range(M) for k in range(K) for t in range(T)),name="couverture des qualifications")
m3.addConstrs(( quicksum([x[i,j,k,tau] for i in range(N) for tau in range(t+1)]) <= wd_per_qual_mat[j,k] for j in range(M) for k in range(K) for t in range(T)),name="unicité de la réalisation d'un projet")
m3.addConstrs((b[j,t] >= r[j,t] for j in range(M) for t in range(T)),name="début d'un projet avant sa fin")
m3.addConstrs((y >= quicksum([s[i,j] for j in range(M)]) for i in range(N)),name="linéarisation de l'objectif 2")
m3.addConstrs((z >= quicksum([b[j,t] - r[j,t] for t in range(T)]) for j in range(M)), name = "linéarisation de l'objectif 3")
m3.addConstrs((r[j,t+1] >= r[j,t] for j in range(M) for t in range(T-1)),name="contrainte interne à la variable r")
m3.addConstrs((b[j,t+1] >= b[j,t] for j in range(M) for t in range(T-1)),name="contrainte interne à la variable b")

m3.setObjective(z,GRB.MINIMIZE)

# Paramétrage (mode mute)
m3.params.outputflag = 0

# Résolution du PL
m3.optimize()

# Affichage en mode texte du PL
print(m3.display())

print("La valeur optimale de l'objectif est f3 = {}".format(m3.objVal))

for i in range(N):
    print('planning de {} : '.format(mapping_names[i]))
    for t in range(T):
        print('jour {} : '.format(t+1),end=' ')
        if vacation_mat[i,t]==0:
                    print('jour de congé')
        for j in range(M):
            for k in range(K):
                if x[i,j,k,t].X == 1 :
                    print('projet {}, compétence {}'.format(j+1,json_instance['qualifications'][k]))

None
La valeur optimale de l'objectif est f3 = 0.0
planning de Olivia : 
jour 1 :  jour 2 :  jour 3 :  jour 4 :  jour 5 :  planning de Liam : 
jour 1 :  jour de congé
jour 2 :  jour 3 :  jour 4 :  jour 5 :  planning de Emma : 
jour 1 :  jour 2 :  jour de congé
jour 3 :  jour 4 :  jour 5 :  